# 15 - Iteradores y generadores: eficiencia y control fino del flujo

## Objetivos de aprendizaje

En esta sesion aprenderas a:

1. Distinguir claramente entre `iterable` e `iterador`.
2. Entender el protocolo de iteracion (`__iter__`, `__next__`, `StopIteration`).
3. Explicar que hace un `for` internamente.
4. Construir iteradores personalizados con clases.
5. Crear generadores con `yield` y razonar sobre su estado.
6. Usar expresiones generadoras para ahorrar memoria.
7. Encadenar transformaciones de forma lazy (perezosa).
8. Aplicar `itertools` para resolver tareas comunes sin reinventar la rueda.
9. Identificar errores sutiles: agotamiento, consumo accidental y efectos secundarios.
10. Resolver problemas donde importa mas el flujo de datos que la sintaxis.


## 1. Modelo mental: iterable vs iterador

Un **iterable** es cualquier objeto que puede producir elementos uno por uno.
Ejemplos: `list`, `tuple`, `str`, `dict`, `set`, `range`, archivos abiertos.

Un **iterador** es el objeto que realmente lleva la cuenta del avance.
- Tiene `__next__()` para entregar el siguiente elemento.
- Cuando no hay mas, lanza `StopIteration`.

Regla practica:
- Iterable = contenedor o fuente.
- Iterador = cursor que avanza sobre esa fuente.


In [ ]:
datos = [10, 20, 30]

it = iter(datos)  # convierte iterable en iterador
print(type(datos))
print(type(it))

print(next(it))
print(next(it))
print(next(it))


## 2. Protocolo de iteracion en detalle

Python usa un contrato simple:

1. `iter(obj)` intenta obtener un iterador desde `obj.__iter__()`.
2. `next(it)` llama `it.__next__()` para pedir el siguiente valor.
3. Si el iterador termino, `__next__()` debe lanzar `StopIteration`.

Este contrato hace que muchos componentes distintos se puedan combinar:
`for`, comprensiones, `sum`, `min`, `max`, `list`, `tuple`, `set`, `dict`, etc.


In [ ]:
texto = "ABC"
it = iter(texto)

while True:
    try:
        ch = next(it)
        print("siguiente:", ch)
    except StopIteration:
        print("iteracion terminada")
        break


## 3. Que hace `for` internamente

`for x in algo:` no "adivina" nada. Sigue el protocolo anterior.
Equivale conceptualmente a:

1. Crear iterador con `iter(algo)`.
2. Pedir elementos con `next(...)` en bucle.
3. Detenerse al recibir `StopIteration`.

Entender esto ayuda a depurar errores cuando mezclas loops, funciones y generadores.


In [ ]:
def for_manual(iterable):
    it = iter(iterable)
    while True:
        try:
            valor = next(it)
        except StopIteration:
            break
        print("procesando", valor)

for_manual(["a", "b", "c"])


## 4. Iteradores personalizados con clases

Crear una clase iteradora te da control explicito sobre:
- Estado interno.
- Condicion de parada.
- Regla para producir cada siguiente valor.

Es util cuando el flujo necesita varias variables de estado o reglas complejas.


In [ ]:
class CuentaRegresiva:
    def __init__(self, inicio):
        self.actual = inicio

    def __iter__(self):
        return self

    def __next__(self):
        if self.actual < 0:
            raise StopIteration
        valor = self.actual
        self.actual -= 1
        return valor

for n in CuentaRegresiva(5):
    print(n)


## 5. Generadores con `yield`

Un generador permite escribir logica de iteracion sin definir una clase completa.

Cada vez que aparece `yield`:
- Se devuelve un valor al consumidor.
- La funcion queda "pausada".
- Al siguiente `next(...)`, se reanuda desde ese punto.

Ventaja clave: codigo compacto y natural para flujos secuenciales.


In [ ]:
def numeros_hasta(limite):
    n = 1
    while n <= limite:
        yield n
        n += 1

for x in numeros_hasta(5):
    print(x)


## 6. El estado vive dentro del generador

Un generador recuerda variables locales entre llamadas.
Eso evita usar estado global o clases cuando no hace falta.

Piensa en un generador como una maquina de estados pequena.


In [ ]:
def acumulados(valores):
    total = 0
    for v in valores:
        total += v
        yield total

print(list(acumulados([3, 1, 4, 2])))


## 7. `return` dentro de un generador

En un generador, `return` no entrega un valor normal al `for`.
Lo que hace es terminar la iteracion lanzando `StopIteration`.

En usos avanzados (consumo manual), ese `return` puede leerse desde `StopIteration.value`.
No es necesario para tareas basicas, pero entenderlo evita confusion.


In [ ]:
def demo_return():
    yield "inicio"
    yield "medio"
    return "fin-logico"

it = iter(demo_return())
print(next(it))
print(next(it))

try:
    next(it)
except StopIteration as e:
    print("StopIteration.value =", e.value)


## 8. Comprension de listas vs expresion generadora

Diferencia central:
- `[f(x) for x in datos]` crea toda la lista de una vez.
- `(f(x) for x in datos)` produce valores bajo demanda.

Si solo vas a recorrer una vez y los datos son grandes, el generador suele ser mejor.
Si necesitas acceso aleatorio o reutilizar resultados muchas veces, la lista puede convenir.


In [ ]:
import sys

numeros = range(100000)
lista = [x * 2 for x in numeros]
gen = (x * 2 for x in numeros)

print("tamano lista:", sys.getsizeof(lista))
print("tamano generador:", sys.getsizeof(gen))

# Ojo: sys.getsizeof(lista) no incluye toda la memoria profunda de objetos anidados,
# pero la diferencia de enfoque (materializar vs producir bajo demanda) sigue siendo real.


## 9. Pipelines lazy: transformar sin cargar todo en memoria

Puedes encadenar pasos para limpiar, filtrar y transformar datos de forma incremental.
Cada paso consume y produce iteradores.

Beneficios:
- Menor memoria.
- Flujo mas claro.
- Mejor composicion de funciones pequenas.


In [ ]:
lineas = [
    "  INFO: inicio  ",
    "DEBUG: detalle interno",
    "WARNING: espacio bajo",
    "INFO: cierre",
]

limpias = (ln.strip() for ln in lineas)
no_debug = (ln for ln in limpias if not ln.startswith("DEBUG"))
mensajes = (ln.split(":", maxsplit=1)[1].strip() for ln in no_debug)

print(list(mensajes))


## 10. Lectura de archivos: iterar linea por linea

En archivos grandes, evita `read()` completo cuando no es necesario.
Iterar por lineas permite procesar streaming y cortar temprano si ya tienes lo que buscabas.


In [ ]:
from io import StringIO

def extraer_enteros(archivo):
    for linea in archivo:
        txt = linea.strip()
        if txt.isdigit():
            yield int(txt)

fake_file = StringIO("10
abc
25

7
")
print(list(extraer_enteros(fake_file)))


## 11. `itertools`: herramientas de alto rendimiento

`itertools` trae bloques listos para trabajar con iteradores.
Algunos muy utiles:
- `islice`: cortar sin materializar todo.
- `chain`: concatenar iterables.
- `count`: contador infinito.
- `takewhile`: tomar mientras una condicion se cumpla.

Usarlos reduce codigo manual y errores.


In [ ]:
from itertools import islice, chain, count, takewhile

print(list(islice(range(100), 5, 12)))
print(list(chain([1, 2], (3, 4), "AB")))
print(list(islice(count(start=10, step=3), 6)))
print(list(takewhile(lambda x: x < 20, count(0, 4))))


## 12. Cuando usar clase iteradora y cuando generador

Usa **generador** cuando:
- La secuencia se expresa bien de forma lineal.
- Quieres menos codigo boilerplate.

Usa **clase iteradora** cuando:
- Necesitas multiples metodos y control detallado del estado.
- Quieres exponer comportamiento adicional (reset, estadisticas, configuracion).

No existe "mejor siempre"; depende del problema.


## 13. Errores comunes y buenas practicas

Errores tipicos:
1. Consumir un iterador y luego asumir que aun tiene datos.
2. Convertir a `list(...)` demasiado pronto (rompe el beneficio lazy).
3. Ocultar efectos secundarios dentro de generadores (dificulta depuracion).
4. Usar nombres ambiguos (`it`, `x`) en pipelines largos.

Buenas practicas:
1. Documenta si una funcion devuelve iterable reutilizable o iterador de un solo uso.
2. Separa pasos del pipeline en funciones cortas con nombres claros.
3. Prueba casos borde: vacio, un elemento, y datos invalidos.
4. Cuando el flujo sea complejo, agrega asserts o logs pequenos.


## 14. Ejercicios de razonamiento cuidadoso

Los siguientes ejercicios estan pensados para obligarte a:
- Razonar sobre estado y consumo.
- Elegir entre materializar vs procesar lazy.
- Detectar bugs que no son de sintaxis, sino de flujo.

Sugerencia: intenta primero sin buscar la solucion.


### Ejercicio 1: De lista a generador sin perder claridad

**Tarea**: Reescribe `cuadrados_pares_lista` como `cuadrados_pares_gen`.

Condiciones:
1. Debe devolver un generador, no lista.
2. Debe procesar bajo demanda.
3. Mantener legibilidad.


In [ ]:
# Tu codigo aqui
# def cuadrados_pares_gen(n):
#     ...


def cuadrados_pares_lista(n):
    return [x * x for x in range(n) if x % 2 == 0]

# Prueba esperada:
# list(cuadrados_pares_gen(10)) == cuadrados_pares_lista(10)


### Ejercicio 2: Iterador por bloques

**Tarea**: Implementa una clase `Bloques` que reciba una secuencia y un tamano de bloque.
Debe iterar devolviendo sublistas de longitud `k` (la ultima puede ser menor).

Ejemplo:
- Entrada: `[1,2,3,4,5]`, `k=2`
- Salida: `[1,2]`, `[3,4]`, `[5]`

Piensa con cuidado como detenerte exactamente cuando corresponde.


In [ ]:
# Tu codigo aqui
# class Bloques:
#     def __init__(self, datos, k):
#         ...
#     def __iter__(self):
#         ...
#     def __next__(self):
#         ...

# Prueba sugerida:
# print(list(Bloques([1, 2, 3, 4, 5], 2)))


### Ejercicio 3: Agotamiento silencioso

**Tarea**: Explica por que este codigo imprime una lista vacia en la segunda conversion.
Luego corrige el flujo sin duplicar trabajo innecesario.


In [ ]:
def fuente():
    for n in range(5):
        print("produciendo", n)
        yield n

it = fuente()
primera = list(it)
segunda = list(it)
print("primera:", primera)
print("segunda:", segunda)

# Tu correccion aqui


### Ejercicio 4: Depurar un pipeline

**Tarea**: Hay un bug logico: el pipeline intenta quedarse con lineas validas,
pero termina descartando casi todo. Detecta el error y corrigelo.

Pista: revisa el orden de `strip`, `split` y la condicion.


In [ ]:
lineas = ["  ok:10", "error", "ok:  20  ", "ok:", " ok:30"]

pipeline = (ln.split(":") for ln in lineas)
pipeline = (partes for partes in pipeline if partes[0] == "ok")
pipeline = (int(partes[1]) for partes in pipeline if partes[1].isdigit())

print(list(pipeline))

# Tu version corregida aqui


### Ejercicio 5: Merge lazy de dos secuencias ordenadas

**Tarea**: Implementa `merge_ordenado(a, b)` que mezcle dos iterables ordenados
sin materializar ambos por completo.

Requisitos:
1. Debe producir elementos en orden ascendente.
2. Debe funcionar aunque uno termine antes.
3. Debe aceptar iterables (no solo listas).


In [ ]:
# Tu codigo aqui
# def merge_ordenado(a, b):
#     ...

# Prueba:
# print(list(merge_ordenado([1,4,8], [2,3,10,11])))


### Ejercicio 6: Ventana deslizante

**Tarea**: Crea un generador `ventanas(iterable, k)` que produzca tuplas de tamano `k`
con ventanas consecutivas.

Ejemplo con `k=3` y `[1,2,3,4,5]`:
- `(1,2,3)`, `(2,3,4)`, `(3,4,5)`

Piensa bien que hacer cuando `k` es mayor que la cantidad de datos.


In [ ]:
# Tu codigo aqui
# def ventanas(iterable, k):
#     ...

# Pruebas sugeridas:
# print(list(ventanas([1,2,3,4,5], 3)))
# print(list(ventanas([1,2], 3)))


### Ejercicio 7: Flatten recursivo con control de strings

**Tarea**: Implementa `aplanar(datos)` para aplanar listas/tuplas anidadas,
pero tratando strings como atomicos (no iterar caracter por caracter).

Ejemplo:
`[1, [2, (3, 4)], "abc", ["xy", [5]]]` -> `1,2,3,4,"abc","xy",5`


In [ ]:
# Tu codigo aqui
# def aplanar(datos):
#     ...

# Prueba:
# datos = [1, [2, (3, 4)], "abc", ["xy", [5]]]
# print(list(aplanar(datos)))


### Ejercicio 8: CSV lazy con validacion

**Tarea**: Dado un iterable de lineas CSV `id,monto`, crea un generador
`leer_montos_validos(lineas)` que entregue solo montos positivos validos (`float`).

Ademas, registra en una lista externa `errores` las lineas rechazadas con su motivo.

Este ejercicio obliga a separar claramente:
1. Parseo.
2. Validacion.
3. Flujo de salida.


In [ ]:
# Tu codigo aqui
# errores = []
# def leer_montos_validos(lineas):
#     ...

# lineas = [
#     "1,100.5",
#     "2,-30",
#     "x,20",
#     "3,abc",
#     "4,40",
# ]
# print(list(leer_montos_validos(lineas)))
# print(errores)


### Ejercicio 9: Round-robin justo

**Tarea**: Implementa `round_robin(*iterables)` que intercale elementos de cada iterable
sin bloquearse cuando uno se agota.

Ejemplo:
`[1,2,3]`, `"AB"`, `[10,20,30,40]` -> `1,"A",10,2,"B",20,3,30,40`

Punto clave: eliminar de forma segura los iteradores ya agotados.


In [ ]:
# Tu codigo aqui
# def round_robin(*iterables):
#     ...

# print(list(round_robin([1,2,3], "AB", [10,20,30,40])))


### Ejercicio 10: Diseno y justificacion tecnica

**Tarea**: Tienes 20 millones de registros de texto y solo necesitas:
1. Contar cuantas lineas cumplen una condicion.
2. Obtener las primeras 50 que fallan validacion.

Escribe dos soluciones:
- Enfoque A materializando en listas.
- Enfoque B lazy con iteradores/generadores.

Luego compara tiempo/memoria de forma razonada y argumenta cual usarias en produccion.


In [ ]:
# Tu propuesta aqui (codigo + comentario tecnico corto)


## 15. Resumen de conceptos clave

1. Iterable no es lo mismo que iterador.
2. `for` depende del protocolo `iter`/`next`.
3. `yield` permite modelar flujos de datos con estado y pausa.
4. Expresiones generadoras y `itertools` ayudan a escalar procesamiento.
5. Pensar en consumo, agotamiento y composicion evita bugs sutiles.


## 16. Reto final opcional

Implementa un mini ETL lazy:
1. Lee lineas simuladas de un log.
2. Filtra solo eventos `VENTA` validos.
3. Convierte montos a `float`.
4. Calcula total, promedio y top 3 montos sin guardar todo si no es necesario.

Objetivo: entrenar el criterio para elegir estructuras de datos y estrategia de iteracion.
